In [2]:
import psycopg2
# Connect to an existing database
conn = psycopg2.connect("dbname='bt' user='postgres' host='localhost' password='severo' port='5432'")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: this creates a new table
cur.execute("select m.tid,m.time,m.edge_id,m.longitude,m.latitude from matched_tracks m order by m.tid,m.time;")
matched_tracks = cur.fetchall()
cur.execute("select distinct edge_id from matched_tracks group by edge_id order by edge_id asc;")
egde_ids = cur.fetchall()
cur.execute("select i.tid,i.start_time,i.end_time,EXTRACT(EPOCH FROM (i.end_time::timestamp - i.start_time::timestamp)) from ib_stops_carlos i order by i.tid, start_time;")
stops = cur.fetchall()
# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
#>>> cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",
#...      (100, "abc'def"))

# Query the database and obtain data as Python objects
#>>> cur.execute("SELECT * FROM test;")
#>>> cur.fetchone()
#(1, 100, "abc'def")

# Make the changes to the database persistent
#>>> conn.commit()

# Close communication with the database
cur.close()

conn.close()

In [3]:
# função que verificar se um ponto é um stop.
def verifica(ponto,stops):
    for stop in stops:
        if ponto[0]==stop[0] and ponto[1]==stop[1]:
            return stop[3]
    return -1  

In [4]:
# Encrontando as trajetorias.
trajetorias = list()
trajetoria = list()
taxi_id = 1

for ponto in matched_tracks:
    if ponto[0]==taxi_id:
        diferenca=verifica([ponto[0],ponto[1]],stops)
        if diferenca==-1:
            trajetoria.append(ponto)
        else:
            trajetoria.append(ponto)
            if diferenca > 120:
                trajetorias.append(trajetoria)
                trajetoria = list()
    else:
        trajetorias.append(trajetoria)
        trajetoria = list()
        trajetoria.append(ponto)
        taxi_id=taxi_id+1     

In [5]:
# Criando dicionário para mapeamento dos egde_ids para os indices da matriz m:
dicionario1=dict()
indice=0
for egde_id in egde_ids:
    dicionario1[egde_id[0]]=indice
    indice=indice+1
# Criando dicionário para o mapeamento dos indices da matriz para os egdes_ids:
dicionario2 = dict()
for i in egde_ids:
    d=dicionario1[i[0]]
    dicionario2[d]=i[0]

In [6]:
# Criando a matriz de transições:
import numpy as np
k=len(dicionario1)
m =np.zeros( (k,k),dtype=np.int32)

In [8]:
# Contando quantas transições exites de cada egde_id para outro:
for traj in trajetorias:
    c = len(traj)
    for i in range(0,c-1):
        if traj[i][2] != traj[i+1][2]:
            d1=dicionario1[traj[i][2]]
            d2=dicionario1[traj[i+1][2]]
            m[d1][d2]=m[d1][d2]+1

In [9]:
# Contando a quantidade de celulas não nulas.
count=0
for n in m:
    for i in n:
        if i!= 0:
            count=count+1

In [10]:
# Calculando as dimensões da matriz para mapeamento.
import math
raiz=math.ceil(math.sqrt(count))
# Criando a matriz para o mapeamento.
n =np.zeros( (raiz,raiz),dtype=np.int32)

In [11]:
# Mapeamento de uma matriz para outra.
dicionario3=dict()
l=0
c=0
for i in range(0,k):
    for j in range(0,k):
        if m[i][j] !=0:
            n[l][c]=m[i][j]
            dicionario3[(l,c)]=(i,j)
            c=c+1
            if c==raiz:
                c=0
                l=l+1

In [12]:
def not_marked(m,a):
    for n in m:
        for i in n:
            if i > 0 and i>=a:
                return True
    return False

def maximo(m,c):
    l=0
    f=0
    for i in range(0,c):
        for j in range(0,c):
            if m[i][j]>m[l][f]:
                l=i
                f=j
    return [l,f]

def marking(a):
    if a>0:
        return -1*a
    return a

def not_marked_2(m,d,f,a,e,c):
    u=-1
    minimo=100000
    for i in range(0,c):
        if m[f][i]>=a:
            t = m[d][f]
            if t<0:
                t=t*(-1)
            if abs(t-m[f][i]) < minimo and abs(t-m[f][i])<=e:
                minimo = abs(t-m[f][i])
                u=i
    return u

def not_marked_3(m,d,f,a,e,c):
    u=-1
    minimo=1000000
    for i in range(0,c):
        if m[i][d]>=a:
            t = m[d][f]
            if t<0:
                t=t*(-1)
            if abs(t-m[u][d])<minimo and abs(t-m[u][d])<=e:
                minimo=abs(t-m[u][d])
                u=i
    return u

In [14]:
def netscan_phase_1(dicionario3,c,dicionario2,m,e,a):
    O=set()
    k=0
    C=list()
    while not_marked(m,a):
        indices=maximo(m,c)
        d=indices[0]
        f=indices[1]
        tupla=dicionario3[(d,f)]
        Sd=dicionario2[tupla[0]]
        Sf=dicionario2[tupla[1]]
        C.append([Sd,Sf])
        m[d][f]=marking(m[d][f])
        u=not_marked_2(m,d,f,a,e,c)
        while u>-1:
            tupla=dicionario3[(f,u)]
            Su=dicionario2[tupla[1]]
            C[k].append(Su)
            m[f][u]=marking(m[f][u])
            d=f
            f=u
            u=not_marked_2(m,d,f,a,e,c)
        u=not_marked_3(m,d,f,a,e,c)
        while u>-1:
            tupla=dicionario3[(u,d)]
            Su=dicionario2[tupla[0]]
            C[k] = [Su]+C[k]
            m[u][d]=marking(m[u][d])
            f=d
            d=u
            u=not_marked_3(m,d,f,a,e,c)
        k=k+1
    return C

In [15]:
C=netscan_phase_1(dicionario3,raiz,dicionario2,n,10,3)

In [19]:
print(trajetorias[0][0])

(1, datetime.datetime(2008, 2, 3, 0, 0, 29), 26934, 116.45539216225, 39.8830124709683)
